# Merging Life Expectancy and Smoking Rate Data

### 1. Loading and Preprocessing the Data

In [1]:
import pandas as pd

In [2]:
life_df = pd.read_csv("Life Expectancy Data.csv")

smoking_df = pd.read_csv("share-of-adults-who-smoke.csv")
smoking_df.rename(columns={"Entity": "Country", 
                           "Share of adults who smoke or use tobacco (age-standardized)": "Smoking rate"
                          }, inplace=True)
smoking_df = smoking_df.drop(columns=['Code'], axis=1)

### 2.  Unifying Country Names

Some countries had different names across various sources. These names were standardized to ensure correct data merging.

In [3]:
country_renames = {
    'Democratic Republic of the Congo': 'Democratic Republic of Congo',
    'Iran (Islamic Republic of)': 'Iran',
    'United Republic of Tanzania': 'Tanzania',
    'Timor-Leste': 'East Timor',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'Micronesia (Federated States of)': 'Micronesia',
    'Viet Nam': 'Vietnam',
    'Republic of Moldova': 'Moldova',
    "Lao People's Democratic Republic": 'Laos',
    'Dominica': 'Dominican Republic',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Cabo Verde': 'Cape Verde',
    "Côte d'Ivoire": "Cote d'Ivoire",
    'Brunei Darussalam': 'Brunei',
    'Russian Federation': 'Russia',
    'United States of America': 'United States',
    'Saint Kitts and Nevis': 'Saint Kitts & Nevis',
    'Saint Vincent and the Grenadines': 'Saint Vincent & the Grenadines',
    'The former Yugoslav republic of Macedonia': 'North Macedonia',
    'Swaziland': 'Eswatini',
    'Republic of Korea': 'South Korea',
    "Democratic People's Republic of Korea": "North Korea"
}

life_df['Country'] = life_df['Country'].replace(country_renames)

### 3. Merging the Datasets

The data was merged based on the Country and Year columns.

In [4]:
data = pd.merge(life_df, smoking_df, on=["Country", "Year"], how="left")

### 4. Filling Missing Values

In [5]:
missing_by_country = data[data['Smoking rate'].isna()]['Country'].value_counts()
print("Countries with completely missing values:")
print([country for country in missing_by_country.index if data[data['Country'] == country]['Smoking rate'].isna().all()])

Countries with completely missing values:
['Djibouti', 'South Sudan', 'North Macedonia', 'Tajikistan', 'Syrian Arab Republic', 'Suriname', 'Central African Republic', 'Sudan', 'Somalia', 'Mozambique', 'Guinea', 'Grenada', 'Gabon', 'Saint Vincent & the Grenadines', 'Equatorial Guinea', 'Eritrea', 'Trinidad and Tobago', 'Nicaragua', 'Libya', 'Angola', 'Antigua and Barbuda', 'Venezuela (Bolivarian Republic of)', 'Micronesia', 'Vanuatu', 'Saint Kitts & Nevis', 'Marshall Islands', 'Niue', 'Monaco', 'Cook Islands', 'Tuvalu', 'Nauru', 'Palau', 'San Marino']


For some African countries, average values for "Africa" were used.

In [14]:
african_missing = [
    'Libya', 'Central African Republic', 'Djibouti', 'Guinea',
    'Somalia', 'Mozambique', 'Sudan', 'Equatorial Guinea',
    'Angola', 'South Sudan', 'Gabon', 'Eritrea'
]

for country in african_missing:
    country_years = data.loc[data['Country'] == country, 'Year']
    for year in country_years:
        africa_value = smoking_df.loc[
            (smoking_df['Country'] == 'Africa') & (smoking_df['Year'] == year),
            'Smoking rate'
        ]
        if not africa_value.empty:
            data.loc[
                (data['Country'] == country) & (data['Year'] == year),
                'Smoking rate'
            ] = africa_value.values[0]

For other countries similar countries or regions were used.

In [16]:
country_fill_map = {
    'Grenada': 'Barbados',
    'Antigua and Barbuda': 'Barbados',
    'Saint Vincent & the Grenadines': 'Saint Lucia',
    'Tajikistan': 'Asia',
    'Vanuatu': 'Oceania',
    'Suriname': 'South America',
    'Micronesia': 'Oceania',
    'Nicaragua': 'South America',
    'North Macedonia': 'Serbia',
    'Trinidad and Tobago': 'Barbados',
    'Venezuela (Bolivarian Republic of)': 'South America',
    'Syrian Arab Republic': 'Asia'
}

for target_country, source_country in country_fill_map.items():
    for year in data[data['Country'] == target_country]['Year'].unique():
        value = smoking_df.loc[
            (smoking_df['Country'] == source_country) &
            (smoking_df['Year'] == year),
            'Smoking rate'
        ]
        if not value.empty:
            data.loc[
                (data['Country'] == target_country) & 
                (data['Year'] == year), 
                'Smoking rate'
            ] = value.values[0]

For countries where data was entirely missing, only data from one year (2015) was available and was inserted for 2013 if no other option was possible.

In [18]:
country_fill_map = {
    'Cook Islands': 'Oceania',
    'Marshall Islands': 'Oceania',
    'Monaco': 'France',
    'Nauru': 'Oceania',
    'Niue': 'New Zealand',
    'Palau': 'Oceania',
    'Saint Kitts & Nevis': 'Saint Lucia',
    'San Marino': 'Italy',
    'Tuvalu': 'Oceania'
}

for target_country, source_country in country_fill_map.items():
    source_value = smoking_df.loc[
        (smoking_df['Country'] == source_country) & 
        (smoking_df['Year'] == 2015),
        'Smoking rate'
    ]
    if not source_value.empty:
        data.loc[
            (data['Country'] == target_country) & 
            (data['Year'] == 2013), 
            'Smoking rate'
        ] = source_value.values[0]

### 5. Interpolation and Final Cleaning

Linear interpolation across years was used to smoothly fill in missing values.

In [21]:
data = data.sort_values(['Country', 'Year'])
data['Smoking rate'] = data.groupby('Country')['Smoking rate'].transform(lambda x: x.interpolate(method='linear'))
data['Smoking rate'] = data.groupby('Country')['Smoking rate'].transform(lambda x: x.bfill().ffill())

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2938 entries, 15 to 2922
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2938 non-null   int64  
 6   Alcohol                          2744 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2385 non-null   float64
 9   Measles                          2938 non-null   int64  
 10   BMI                             2904 non-null   float64
 11  under-five deaths                2938 non-null   int64  
 12  Polio                   

In [23]:
data.columns = data.columns.str.strip().str.title().str.replace(' ', '_')
data.columns = [col.upper() if col in ['Bmi', 'Hiv/Aids', 'Gdp'] else col for col in data.columns]

In [24]:
data.columns

Index(['Country', 'Year', 'Status', 'Life_Expectancy', 'Adult_Mortality',
       'Infant_Deaths', 'Alcohol', 'Percentage_Expenditure', 'Hepatitis_B',
       'Measles', 'BMI', 'Under-Five_Deaths', 'Polio', 'Total_Expenditure',
       'Diphtheria', 'HIV/AIDS', 'GDP', 'Population', 'Thinness__1-19_Years',
       'Thinness_5-9_Years', 'Income_Composition_Of_Resources', 'Schooling',
       'Smoking_Rate'],
      dtype='object')

In [ ]:
data.to_csv('life_expectancy_data.csv', index=False)